In [1]:
# use directly movies and ratings tables from moviedb which are external tables

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("MovieLensSQL")

config.set("spark.local.dir", "/home/ubuntu/spark-temp")

# while using hive.metastore.warehouse.dir, we should not use spark warehouse dir

config.set("hive.metastore.uris", "thrift://localhost:9083")
config.set("hive.metastore.warehouse.dir", "hdfs://localhost:9000/user/hive/warehouse")


from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .enableHiveSupport()\
                    .getOrCreate()

sc = spark.sparkContext

In [3]:
# hdfs dfs -ls /movies
# hdfs dfs -ls /ratings

In [10]:
# external table pointing to csv file(s)
spark.sql("""
SELECT * FROM moviedb.movies LIMIT 5

""").show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+



In [3]:
# moviedb.ratings is external table
spark.sql("""
SELECT * FROM moviedb.ratings LIMIT 5
""").show()

+-------+--------+------+----------------+
|user_id|movie_id|rating|rating_timestamp|
+-------+--------+------+----------------+
|   null|    null|  null|            null|
|      1|       1|     4|       964982703|
|      1|       3|     4|       964981247|
|      1|       6|     4|       964982224|
|      1|      47|     5|       964983815|
+-------+--------+------+----------------+



In [10]:
# Rating Analytics for popular movies

popularMoviesDf = spark.sql("""
SELECT movie_id, avg(rating) as avg_rating, count(user_id) as total_ratings
FROM moviedb.ratings
WHERE movie_id IS NOT null
GROUP BY movie_id
""")

popularMoviesDf.printSchema()
popularMoviesDf.show(5)

root
 |-- movie_id: integer (nullable = true)
 |-- avg_rating: decimal(14,4) (nullable = true)
 |-- total_ratings: long (nullable = false)

+--------+----------+-------------+
|movie_id|avg_rating|total_ratings|
+--------+----------+-------------+
|    1580|    3.6364|          165|
|    2366|    3.7200|           25|
|    3175|    3.6933|           75|
|    1088|    3.5714|           42|
|   32460|    4.5000|            4|
+--------+----------+-------------+
only showing top 5 rows



In [11]:
# Rating Analytics for popular movies
# create a temporary view out of SQL SELECT statement
# CTAS - CREATE TABLE AS, the popular_movies shall have output analytical query
spark.sql("""
CREATE OR REPLACE TEMP VIEW popular_movies AS
SELECT movie_id, avg(rating) as avg_rating, count(user_id) as total_ratings
FROM moviedb.ratings
WHERE movie_id IS NOT null
GROUP BY movie_id
HAVING avg_rating >= 3.5 AND total_ratings >= 100
""")

DataFrame[]

In [12]:
spark.sql("SHOW TABLES").show()

+--------+--------------+-----------+
|database|     tableName|isTemporary|
+--------+--------------+-----------+
| default|        brands|      false|
| default|     employees|      false|
| default|    employees2|      false|
| default|      payroles|      false|
|        |popular_movies|       true|
+--------+--------------+-----------+



In [24]:
spark.sql("SELECT * FROM popular_movies").show(5)

+-------+------------------+-------------+
|movieId|        avg_rating|total_ratings|
+-------+------------------+-------------+
|    858|         4.2890625|          192|
|   1270| 4.038011695906433|          171|
|   1265| 3.944055944055944|          143|
|    588|3.7923497267759565|          183|
|    296| 4.197068403908795|          307|
+-------+------------------+-------------+
only showing top 5 rows



In [19]:
# do not run 
# spark.sql("DROP  VIEW popular_movies")

DataFrame[]

In [13]:
# join to get the movie titles
# INNER JOIN moviedb.movies movies  IN THIS STATEMENT, movies is ALAIS
spark.sql("""
SELECT movies.movie_id, title, avg_rating, total_ratings 
FROM popular_movies 
INNER JOIN moviedb.movies movies ON popular_movies.movie_id = movies.movie_id
ORDER BY avg_rating DESC
""").show()

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|     318|"Shawshank Redemp...|    4.5300|          317|
|   48516|           "Departed|    4.4673|          107|
|    2959|   Fight Club (1999)|    4.4404|          218|
|   58559|        "Dark Knight|    4.4295|          149|
|    1213|   Goodfellas (1990)|    4.3968|          126|
|     858|          "Godfather|    4.3958|          192|
|    2329|American History ...|    4.3876|          129|
|    1197|     "Princess Bride|    4.3732|          142|
|      50|     "Usual Suspects|    4.3725|          204|
|    1221| "Godfather: Part II|    4.3721|          129|
|     912|   Casablanca (1942)|    4.3600|          100|
|    7361|Eternal Sunshine ...|    4.3511|          131|
|     527|Schindler's List ...|    4.3500|          220|
|    1193|One Flew Over the...|    4.3308|          133|
|     260|Star Wars: Episod...|

In [14]:
 # CTAS - CREATE TEMP VIEW AS SELECT
# FIXME
spark.sql("""
CREATE OR REPLACE TEMP VIEW most_popular_movies AS 
SELECT movies.movie_id, title, avg_rating, total_ratings 
FROM popular_movies 
INNER JOIN moviedb.movies movies ON popular_movies.movie_id = movies.movie_id
ORDER BY avg_rating DESC
""")

DataFrame[]

In [21]:
# CREATE A PERMANENT MANAGED TABLE in HIVE CATALOG
# DATA SHALL BE STORED IN /user/hive/warehouse/moviedb.db/popular_movies
# CTAS - CREATE TABLE AS SELECT
# FIXME
spark.sql("""
CREATE TABLE moviedb.most_popular_movies6 AS 
SELECT movies.movie_id, title, avg_rating, total_ratings 
FROM popular_movies 
INNER JOIN moviedb.movies movies ON popular_movies.movie_id = movies.movie_id
ORDER BY avg_rating DESC
""")

DataFrame[]

In [15]:
spark.sql("SELECT * FROM most_popular_movies").show(5)

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|     318|"Shawshank Redemp...|    4.5300|          317|
|   48516|           "Departed|    4.4673|          107|
|    2959|   Fight Club (1999)|    4.4404|          218|
|   58559|        "Dark Knight|    4.4295|          149|
|    1213|   Goodfellas (1990)|    4.3968|          126|
+--------+--------------------+----------+-------------+
only showing top 5 rows



In [16]:
# get dataframe from table/temp view

mostPopularMoviesDf = spark.table("most_popular_movies")

mostPopularMoviesDf.write\
                    .mode('overwrite')\
                    .saveAsTable("moviedb.most_popular_movies2")

mostPopularMoviesDf.show(5)

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|     318|"Shawshank Redemp...|    4.5300|          317|
|   48516|           "Departed|    4.4673|          107|
|    2959|   Fight Club (1999)|    4.4404|          218|
|   58559|        "Dark Knight|    4.4295|          149|
|    1213|   Goodfellas (1990)|    4.3968|          126|
+--------+--------------------+----------+-------------+
only showing top 5 rows



In [17]:
spark.sql("SELECT * FROM moviedb.most_popular_movies2").show(5)

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|    4896|Harry Potter and ...|    3.9533|          107|
|    1198|Raiders of the Lo...|    4.3200|          200|
|     293|Léon: The Profess...|    4.1429|          133|
|    6539|Pirates of the Ca...|    3.9664|          149|
|     253|Interview with th...|    3.5596|          109|
+--------+--------------------+----------+-------------+
only showing top 5 rows



In [22]:
spark.sql("SHOW TABLES IN moviedb").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| moviedb| most_popular_movies|      false|
| moviedb|most_popular_movies2|      false|
| moviedb|most_popular_movies6|      false|
| moviedb|              movies|      false|
| moviedb|             ratings|      false|
| moviedb|             reviews|      false|
|        | most_popular_movies|       true|
|        |      popular_movies|       true|
+--------+--------------------+-----------+



In [ ]:
# On Hive cli

"""
select * from moviedb.most_popular_movies;
"""